In [1]:
from osgeo import gdal, ogr, osr
from osgeo import gdal
import numpy as np

In [7]:


# Open the TIF file
dataset = gdal.Open(r'E:\wenqu\aviris\arctic_region\site7\output\ang20190713t023324_rfl_v2v2_img_brdf')

# Select the bands
red_band = dataset.GetRasterBand(55)
nir_band = dataset.GetRasterBand(81)

# Read the bands as arrays
red = red_band.ReadAsArray()
nir = nir_band.ReadAsArray()

# Calculate NDVI
ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)

# Handle division by zero
ndvi[np.isnan(ndvi)] = 0

# Create a new TIF file for output
driver = gdal.GetDriverByName('GTiff')
out_dataset = driver.Create('E:/wenqu/aviris/arctic_region/NDVI/site7_ndvi.tif', dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Float32)

# Write NDVI to the new TIF file
out_band = out_dataset.GetRasterBand(1)
out_band.WriteArray(ndvi)

# Set georeference and projection from the original file
out_dataset.SetGeoTransform(dataset.GetGeoTransform())
out_dataset.SetProjection(dataset.GetProjection())

# Save and close the datasets
out_band.FlushCache()
out_dataset.FlushCache()


In [2]:
# 打开 TIF 文件
dataset = gdal.Open('E:/wenqu/aviris/arctic_region/NDVI/site4bc_ndvi.tif', gdal.GA_ReadOnly)

# 获取第一个波段（通常 TIF 文件至少包含一个波段）
band = dataset.GetRasterBand(1)

# 将波段读为数组
array = band.ReadAsArray()

# 重新分类：小于 0 的设置为 0，大于等于 0 的设置为 1
array = np.where(array < 0.15, 0, 1)

# 创建新的 TIF 文件
driver = gdal.GetDriverByName('GTiff')
out_dataset = driver.Create('E:/wenqu/aviris/arctic_region/NDVI/mask/site4bc_ndvi_mask_update.tif', band.XSize, band.YSize, 1, gdal.GDT_Byte)

# 写入重新分类后的数组
out_band = out_dataset.GetRasterBand(1)
out_band.WriteArray(array)

# 设置地理变换和投影
out_dataset.SetGeoTransform(dataset.GetGeoTransform())
out_dataset.SetProjection(dataset.GetProjection())

# 清除缓存并关闭文件
out_band.FlushCache()
out_dataset.FlushCache()